In [20]:
import time
import os
import paramiko
from io import StringIO
import time
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from IPython.display import clear_output
import datetime

In [21]:
# SSH credentials and remote CSV file path
# host = 'lxslc7.ihep.ac.cn'
# username = os.environ.get('IHEP_USERNAME')
# password = os.environ.get('IHEP_PASSWORD')
# remote_csv_file = '/hpcfs/cepc/higgs/frank/GNN/ds/output_tag/summaries_0.csv'
# remote_csv_file = '/hpcfs/cepc/higgs/frank/GNN/ds/output_rec/summaries_0.csv'


host = 'bl-0.inpac.sjtu.edu.cn'
username = os.environ.get('SJTU_USERNAME')
password = os.environ.get('SJTU_PASSWORD')
remote_csv_file = '/lustre/collider/zhangyulei/DeepLearning/Tracking/workspace/output/summaries_0.csv'

# Establish an SSH connection
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, username=username, password=password)
print(f'Successfully connected to {username}@{host}')

Successfully connected to zhangyulei@bl-0.inpac.sjtu.edu.cn


In [22]:
def update_csv(remote_file: str = None):
    sftp = ssh.open_sftp()

    if remote_file is None:
        remote_file = remote_csv_file

    with sftp.open(remote_file, 'r') as file:
        remote_csv_data = file.read().decode('utf-8')
        timestamp = sftp.stat(remote_file).st_mtime

        dt_object = datetime.datetime.fromtimestamp(timestamp)

        return pd.read_csv(StringIO(remote_csv_data)), \
            f"{dt_object.year} - {dt_object.month} - {dt_object.day}   {dt_object.hour} : {dt_object.minute}"


def update_fig(df, t):
    fig = go.Figure()

    y_max, y_min = df.max()[['train_loss', 'valid_loss']].max(), df.min()[['train_loss', 'valid_loss']].min()

    df_new = df[['epoch', 'train_loss', 'valid_loss', 'valid_dp_mean', 'valid_dp_std', 'valid_acc']]. \
        groupby('epoch').transform("mean").drop_duplicates(keep='last', subset=['train_loss'])

    mu_max = df.abs().max()['valid_dp_mean']
    std_max = df.abs().max()['valid_dp_std']

    fig.data = []
    fig.add_trace(
        go.Scatter(
            x=df_new.index,
            y=df_new['valid_dp_mean'],
            mode='lines', name="itr: p mean",
            line=dict(dash='dot', color="#ff7f0e"), yaxis='y3',
            legendgroup="Momentum",
            legendgrouptitle_text="Momentum",
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df_new.index,
            y=df_new['valid_dp_std'],
            mode='lines', name="itr: p std",
            line=dict(dash='dot', color="#64a05f"), yaxis='y4',
            legendgroup="Momentum",
        )
    )

    fig.add_trace(
        go.Scatter(
            x=df.index, y=df['train_loss'], mode='lines', name="itr: train", legendgroup="Itr Loss",
            legendgrouptitle_text="Itr Loss", line=dict(dash='dot'))
    )
    fig.add_trace(
        go.Scatter(
            x=df.index, y=df['valid_loss'], mode='lines', name="itr: valid", legendgroup="Itr Loss",
            line=dict(dash='dot')
        ))
    fig.add_trace(
        go.Scatter(
            x=df_new.index,
            y=-np.log10(1 - df_new['valid_acc']),
            # y=df['valid_acc'],
            mode='lines', name="itr: accuracy",
            line=dict(dash='dot', color="#11ADF0"), yaxis='y2',
            legendgroup="Edge Label",
            legendgrouptitle_text="Edge Label",
        )
    )

    fig.add_trace(go.Scatter(
        x=df_new.index, y=df_new['train_loss'], mode='lines+markers', name="epoch: train",
        legendgroup="Epoch Loss", legendgrouptitle_text="Epoch Loss"
    ))
    fig.add_trace(go.Scatter(
        x=df_new.index, y=df_new['valid_loss'], mode='lines+markers', name="epoch: valid",
        legendgroup="Epoch Loss"
    ))

    for lr in df.drop_duplicates(keep="first", subset=['lr'])['lr'].items():
        print(lr)
        fig.add_vline(x=lr[0], line_width=2, line_dash="dash", line_color="grey")
        fig.add_annotation(
            text=f'$\eta = {lr[1]}$',
            x=lr[0] + 0.5, y=np.log10(y_min) * 1.02,  # Set the position using numeric values
            xanchor='left',
            # yanchor='bottom',
            showarrow=False,
            font=dict(size=14, color='grey')
        )

    fig.update_layout(
        title=f"{t} --> Epoch: {df['epoch'].iloc[-1]}, Iteration: {df['itr'].iloc[-1]}",
        width=1200,
        height=700,
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        xaxis=dict(
            title_text="Total iteration",
            showgrid=False,
            mirror=True,
            showline=True,
            zeroline=False,
            linewidth=2,
            linecolor='#666666', gridcolor='#d9d9d9',
            domain=[0.1, 0.85],
        ),
        yaxis=dict(
            title=r"Loss",
            titlefont=dict(color="#d62728"),
            tickfont=dict(color="#d62728"),
            showgrid=False,
            linecolor="#d62728", gridcolor='#d9d9d9',
            zeroline=False,
            type="log",
            range=[np.log10(y_min) * 1.05, np.log10(y_max) * 1.05],
        ),
        yaxis2=dict(
            # title=r"$\textrm{95% CI }{\Large \kappa_{2V}}\textrm{ Interval}$",
            title=r"$-log(\epsilon_{error})$",
            titlefont=dict(color="#11ADF0"),
            tickfont=dict(color="#11ADF0"),
            anchor="free",
            overlaying="y",
            side="left",
            position=0.0,
            showgrid=False,
            linecolor="#11ADF0", gridcolor='#d9d9d9',
            zeroline=False,
            # type="log",
            # range=[3.0, 3.7],
        ),
        yaxis3=dict(
            title=r"$\mu^{p}_{\textrm{valid}}$",
            titlefont=dict(color="#ff7f0e"),
            tickfont=dict(color="#ff7f0e"),
            anchor="x",
            overlaying="y",
            side="right",
            showgrid=False,
            linecolor="#ff7f0e", gridcolor='#d9d9d9',
            zeroline=False,
            range=[-mu_max * 1.05, mu_max * 1.05]
        ),
        yaxis4=dict(
            title=r"$\sigma^{p}_{\textrm{valid}}$",
            titlefont=dict(color="#64a05f"),
            tickfont=dict(color="#64a05f"),
            anchor="free",
            overlaying="y",
            side="right",
            position=0.95,
            showgrid=False,
            linecolor="#64a05f", gridcolor='#d9d9d9',
            zeroline=False,
            # range=[0.0, std_max],
            range=[0.0, 1.0],
        ),
    )

    return fig

In [24]:
tag = '/hpcfs/cepc/higgs/frank/GNN/ds/output_tag/summaries_0.csv'
rec = '/hpcfs/cepc/higgs/frank/GNN/ds/output_rec/summaries_0.csv'

# tar = tag
tar = rec

# df, t = update_csv('/hpcfs/cepc/higgs/frank/GNN/ds/output_rec.gradnorm.4.newloss/summaries_0.csv')
# df, t = update_csv('/hpcfs/cepc/higgs/frank/GNN/ds/output_rec.gradnorm.4/summaries_0.csv')
df, t = update_csv('/lustre/collider/zhangyulei/DeepLearning/Tracking/workspace/output.rec.new/summaries_0.csv')
fig = update_fig(df, t)
fig.show()
#
# while True:
#     df, t = update_csv(tar)
#     fig = update_fig(df, t)
#     time.sleep(120)
#     clear_output(wait=False)
#     fig.show()

(0, 0.01)
(5, 0.0141421356237309)


In [6]:
df_new = df[[
    'epoch',
    'valid_num_tracks_diff_0',
    'valid_num_tracks_diff_1',
    'valid_num_tracks_diff_2',
    'valid_num_tracks_diff_-1',
    'valid_num_tracks_diff_-2',
    'valid_num_tracks_diff_overflow',
    'valid_num_tracks_diff_underflow',
    'valid_TP',
    'valid_TN',
    'valid_FP',
    'valid_FN',
]].groupby('epoch', as_index=False).transform("sum").drop_duplicates(keep='last')

In [7]:
def update_stack(df: pd.DataFrame, columns, t):
    fig = go.Figure()

    norm_sum = df[columns].sum(axis=1)

    for key in columns:
        fig.add_trace(go.Bar(
            name=key,
            y=df[key]/norm_sum,
            x=df['epoch'],
            width=1,
            offset=0,
        ))

    fig.update_layout(
        title=f"{t} --> Epoch: {df['epoch'].iloc[-1]}",
        width=800,
        height=700,
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        barmode="stack"
    )

    fig.show()

update_stack(df_new, [col for col in df_new.columns if 'valid_num_tracks_diff' in col], t)

update_stack(df_new, [col for col in df_new.columns if 'valid_num_tracks_diff' not in col][1:], t)


In [26]:
df

,lr,train_loss,l1,l2,grad_norm,train_batches,itr,epoch,valid_loss,valid_acc,...,valid_sum_total,valid_dp_mean,valid_dp_std,valid_num_tracks_diff_0,valid_num_tracks_diff_1,valid_num_tracks_diff_2,valid_num_tracks_diff_-1,valid_num_tracks_diff_-2,valid_num_tracks_diff_underflow,valid_num_tracks_diff_overflow
0,0.001000,0.819160,3211.462059,26.683606,3.883361,109,0,0,0.456399,0.853119,...,1220358,-999.0,NaN,126,10,1,945,1854,45,0
1,0.001000,0.357289,3228.623506,26.641615,5.903439,109,1,0,0.319525,0.895994,...,1159147,-999.0,NaN,215,29,4,897,1760,73,1
2,0.001000,0.322738,3264.398823,26.667085,4.439903,109,2,0,0.283798,0.919074,...,1023704,-999.0,NaN,201,31,2,910,1724,104,0
3,0.001000,0.284309,3288.573569,26.678092,4.024795,109,3,0,0.248169,0.936032,...,1124272,-999.0,NaN,144,17,1,950,1836,37,0
4,0.001000,0.348330,3325.284223,26.743483,8.441125,108,4,0,0.463225,0.811314,...,806354,-999.0,NaN,193,23,3,1128,1587,26,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,0.000028,0.015512,4653.974381,33.996547,0.467806,109,0,74,0.032607,0.996522,...,1220358,-999.0,NaN,1397,45,0,1103,403,33,0
371,0.000028,0.014296,4653.819907,33.995725,0.632091,109,1,74,0.022936,0.996546,...,1159147,-999.0,NaN,1972,61,1,744,156,45,0
372,0.000028,0.022103,4653.794625,33.995504,0.360078,109,2,74,0.051008,0.994858,...,1023704,-999.0,NaN,2207,73,2,514,127,48,1
373,0.000028,0.022851,4654.033777,33.996706,1.271445,109,3,74,0.036505,0.994691,...,1124272,-999.0,NaN,397,17,1,1487,1045,38,0
